In [86]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e8/sample_submission.csv
/kaggle/input/playground-series-s5e8/train.csv
/kaggle/input/playground-series-s5e8/test.csv


In [87]:
train = pd.read_csv('/kaggle/input/playground-series-s5e8/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s5e8/test.csv')

In [88]:
train

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,42,technician,married,secondary,no,7,no,no,cellular,25,aug,117,3,-1,0,unknown,0
1,1,38,blue-collar,married,secondary,no,514,no,no,unknown,18,jun,185,1,-1,0,unknown,0
2,2,36,blue-collar,married,secondary,no,602,yes,no,unknown,14,may,111,2,-1,0,unknown,0
3,3,27,student,single,secondary,no,34,yes,no,unknown,28,may,10,2,-1,0,unknown,0
4,4,26,technician,married,secondary,no,889,yes,no,cellular,3,feb,902,1,-1,0,unknown,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749995,749995,29,services,single,secondary,no,1282,no,yes,unknown,4,jul,1006,2,-1,0,unknown,1
749996,749996,69,retired,divorced,tertiary,no,631,no,no,cellular,19,aug,87,1,-1,0,unknown,0
749997,749997,50,blue-collar,married,secondary,no,217,yes,no,cellular,17,apr,113,1,-1,0,unknown,0
749998,749998,32,technician,married,secondary,no,-274,no,no,cellular,26,aug,108,6,-1,0,unknown,0


train df observations:
- shape : (750000, 18)
- nulls: 0 for all columns
- -1 as pdays means they have never been contacted before

In [89]:
#some feature engineering
train['was_contacted'] = (train['pdays'] != -1).astype(int) #a column to check if a specific customer was ever contacted or not
test['was_contacted'] = (test['pdays'] != -1).astype(int)

In [90]:
categorical_columns = ['job', 'marital', 'education', 'contact', 'month', 'poutcome']
binary_columns = ['default', 'housing', 'loan', 'was_contacted']
num_columns = []

# **NOTE TO SELF: WHEN PREPROCESSING BINARY COLUMNS, DO NOT USE ENCODERS, BUT RATHER JUST TRANSFORM TO 0 & 1 AND TREAT AS NUMERICAL# **

In [91]:
#preprocessing categorical columns
from sklearn.preprocessing import OneHotEncoder

def preprocess_categories(df):
    encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
    transformed = encoder.fit_transform(df[categorical_columns])
    
    encoded_df = pd.DataFrame(transformed, 
                              columns=encoder.get_feature_names_out(categorical_columns),
                              index=df.index)
    
    df = df.drop(columns=categorical_columns)
    
    df = pd.concat([df, encoded_df], axis=1)
    
    return df

train = preprocess_categories(train)

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [92]:
#preprocessing binary columns

def yes_or_no(binary_series):
    return binary_series.apply(lambda x: 1 if x == 'yes' else 0 if x == 'no' else x)

def preprocess_binary(df):
    for col in binary_columns:
        df[col] = yes_or_no(df[col])
    return df

train = preprocess_binary(train)

In [95]:
import lightgbm as lgb
from xgboost import XGBClassifier

model2 = XGBClassifier(use_label_encoder=False, eval_metric='logloss')


model = lgb.LGBMClassifier(
    num_leaves=31,
    learning_rate=0.05,
    n_estimators=500,
    random_state=42
)
x_train = train.drop(columns = ['id', 'y'])
y_train = train['y']

In [ ]:
model.fit(x_train, y_train)
model2.fit(x_train, y_train)

[LightGBM] [Info] Number of positive: 90488, number of negative: 659512
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.053913 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1041
[LightGBM] [Info] Number of data points in the train set: 750000, number of used features: 49
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.120651 -> initscore=-1.986283
[LightGBM] [Info] Start training from score -1.986283


In [ ]:
test

In [ ]:
test = preprocess_categories(test)
test = preprocess_binary(test)

In [ ]:
x_test = test.drop(columns = ['id'])
y_pred = (lgb_model.predict_proba(X_test)[:,1] + xgb_model.predict_proba(X_test)[:,1]) / 2
y_pred = y_pred[:, 1]

In [ ]:
submission = pd.DataFrame({
    'id' : test['id'],
    'y' : y_pred
})

In [ ]:
submission.to_csv('submission.csv', index = False)